Content
Each record consists of three attributes:
- is_sarcastic: 1 if the record is sarcastic otherwise 0
- headline: the headline of the news article
- article_link: link to the original news article. Useful in collecting supplementary data

Reference :- https://github.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection

Required Files given in below link.

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk



Get files

In [47]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [48]:
project_path = "/content/drive/My Drive/Colab Notebooks/Sequence Model in NLP/"

Read Data

In [49]:
import pandas as pd
df = pd.read_json(project_path + "Sarcasm_Headlines_Dataset.json", lines=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


2. Drop one column (article_link)

In [50]:
df = df.drop(['article_link'], axis=1)
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


Length of each headline

In [51]:
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))
df.head()

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14
3,"boehner just wants wife to listen, not come up...",1,13
4,j.k. rowling wishes snape happy birthday in th...,0,11


Installing Tensorflow2.0

In [52]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.5.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? n


In [53]:
!pip install tensorflow==2.5.0

Import required modules

In [54]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

In [55]:
max_features = 10000
maxlen = 25
embedding_size = 200

Apply Keras Tokenizer and get indices for words

In [39]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
print("Number of Samples:", len(X))
print(X[0])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[307, 678, 3336, 2297, 47, 381, 2575, 5, 2576, 8433]
Number of Labels:  26709
0


Vocabulary size

In [56]:
num_words = len(tokenizer.word_index) + 1
print(num_words)

29657


Get Glove Word Embeddings

In [58]:
glove_file = project_path + "glove.6B.zip"

In [59]:
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

In [60]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [61]:
len(embeddings.values())

400000

Model

In [62]:
model = Sequential()
model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 5
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/5
214/214 [==============================] - 52s 225ms/step - loss: 0.4778 - accuracy: 0.7711 - val_loss: 0.3565 - val_accuracy: 0.8428
Epoch 2/5
214/214 [==============================] - 47s 221ms/step - loss: 0.2832 - accuracy: 0.8919 - val_loss: 0.3320 - val_accuracy: 0.8627
Epoch 3/5
214/214 [==============================] - 48s 223ms/step - loss: 0.1922 - accuracy: 0.9304 - val_loss: 0.3419 - val_accuracy: 0.8594
Epoch 4/5
214/214 [==============================] - 48s 222ms/step - loss: 0.1359 - accuracy: 0.9517 - val_loss: 0.5178 - val_accuracy: 0.8585
Epoch 5/5
214/214 [==============================] - 48s 223ms/step - loss: 0.0893 - accuracy: 0.9701 - val_loss: 0.5963 - val_accuracy: 0.8584
